# Register model and deploy locally with advanced usages

This example shows how to deploy a web service in step-by-step fashion:

 1. Register model
 2. Deploy the image as a web service in a local Docker container.
 3. Quickly test changes to your entry script by reloading the local service.
 4. Optionally, you can also make changes to model, conda or extra_docker_file_steps and update local service

## Prerequisites
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the [configuration](../../../configuration.ipynb) Notebook first if you haven't.

In [1]:
import os

subscription_id = os.getenv("40bffbcc-578f-4e44-bd6d-972552eb6513", default="40bffbcc-578f-4e44-bd6d-972552eb6513")
resource_group = os.getenv("gestaltml", default="gestaltml")
workspace_name = os.getenv("fastai2", default="fastai2")
workspace_region = os.getenv("eastus", default="eastus")

score_script = 'score_local.py'
model_name = 'breastcancerfastai_local03'
service_name = 'breastcancerlocal03'

In [2]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.13.0


## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [3]:
from azureml.core import Workspace

ws = Workspace.get(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group)

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

fastai2
gestaltml
eastus
40bffbcc-578f-4e44-bd6d-972552eb6513


## Create trained model

For this example, we will train a small model on scikit-learn's [diabetes dataset](https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset). 

In [4]:
# import joblib

# from sklearn.datasets import load_diabetes
# from sklearn.linear_model import Ridge

# dataset_x, dataset_y = load_diabetes(return_X_y=True)

# sk_model = Ridge().fit(dataset_x, dataset_y)

# joblib.dump(sk_model, "sklearn_regression_local.pkl")

## Register Model

You can add tags and descriptions to your models. we are using `sklearn_regression_model.pkl` file in the current directory as a model with the name `sklearn_regression_model` in the workspace.

Using tags, you can track useful information such as the name and version of the machine learning library used to train the model, framework, category, target customer etc. Note that tags must be alphanumeric.

In [5]:
from azureml.core.model import Model

model = Model.register(model_path="export.pkl",
                       model_name=model_name,
                       tags={'area': "breast", 'type': "transfer-learning", 'classes':'Positive,Negative'},
                       description="Breast Cancer Diag-embeded X,Y functions:092220",
                       workspace=ws)

Registering model breastcancerfastai_local03


## Open Existing Model<br>
Use if model already installed in Azure ML Workspace

In [ ]:
# from azureml.core import Model
# from azureml.core.resource_configuration import ResourceConfiguration

# model = Model(ws, model_name)

# print('Name:', model.name)
# print('Version:', model.version)

Manage your dependencies in a folder

In [6]:
import os

source_directory = "source_directory"

os.makedirs(source_directory, exist_ok=True)
os.makedirs(os.path.join(source_directory, "x/y"), exist_ok=True)
os.makedirs(os.path.join(source_directory, "env"), exist_ok=True)
os.makedirs(os.path.join(source_directory, "dockerstep"), exist_ok=True)

Display scoring file (var score_script)

In [7]:
with open(score_script) as f:
    print(f.read())


import os, base64
import torch
import json
from azureml.core.model import Model
from azureml.core import Workspace
import fastai 
# from fastai.basics import *
from fastai.vision.all import *
from fastai.callback.all import *
# from fastai.metrics import accuracy 
# from fastai.metrics import error_rate
import urllib.request

    # global model

    # # The AZUREML_MODEL_DIR environment variable indicates
    # # a directory containing the model file you registered.
    # model_filename = 'sklearn_regression_model.pkl'
    # model_path = os.path.join(os.environ['AZUREML_MODEL_DIR'], model_filename)

    # model = joblib.load(model_path)

def init():
    global learner
    # The AZUREML_MODEL_DIR environment variable indicates a directory containing the model file you registered.  
    #this init works 
    pathsrc=Path()

    model_path=os.getenv('AZUREML_MODEL_DIR')  
    
    filename="export.pkl"
    classes = ['Positive','Negative']
    model_path_file = os.path.join(os.environ['AZ

In [ ]:
# %%writefile source_directory/extradata.json
# {
#     "people": [
#         {
#             "website": "microsoft.com", 
#             "from": "Seattle", 
#             "name": "Mrudula"
#         }
#     ]
# }

## Create Inference Configuration

 - file_path: input parameter to Environment constructor. Manages conda and python package dependencies.
 - env.docker.base_dockerfile: any extra steps you want to inject into docker file
 - source_directory: holds source path as string, this entire folder gets added in image so its really easy to access any files within this folder or subfolder
 - entry_script: contains logic specific to initializing your model and running predictions

In [8]:
#Create environment

from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.conda_dependencies import CondaDependencies

entryscript_fullpath = "x/y/" + str(score_script)

myenv = Environment(model_name)
myenv.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults~=1.13.0', 'fastcore==1.0.0', 'fastprogress==1.0.0', 'fastscript==1.0.0', 'Pillow==5.4.1', 'requests', 'torch==1.6.0', 'torchvision>=0.5.0', 'fastai==2.0.6', 'ipython'
])
# explicitly set base_image to None when setting base_dockerfile
myenv.docker.base_image = None
myenv.docker.base_dockerfile = "FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04\nRUN echo \"this is test\""
myenv.inferencing_stack_version = "latest"

In [9]:
#Create inference config
inference_config = InferenceConfig(source_directory=source_directory,                                 entry_script=entryscript_fullpath,
    environment=myenv)

## Deploy Model as a Local Docker Web Service

*Make sure you have Docker installed and running.*

Note that the service creation can take few minutes.

NOTE:

The Docker image runs as a Linux container. If you are running Docker for Windows, you need to ensure the Linux Engine is running:

    # PowerShell command to switch to Linux engine
    & 'C:\Program Files\Docker\Docker\DockerCli.exe' -SwitchLinuxEngine

In [10]:
from azureml.core.webservice import LocalWebservice

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

local_service.wait_for_deployment()

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 84659b36a99e4a16a8757ae52b4288dc.azurecr.io
Logging into Docker registry 84659b36a99e4a16a8757ae52b4288dc.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 84659b36a99e4a16a8757ae52b4288dc.azurecr.io/azureml/azureml_e5a6bc09d9ffd8bce12ea2b9ad46d390
 ---> 7a313ce8cfc0
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 7800a2bb5239
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjQwYmZmYmNjLTU3OGYtNGU0NC1iZDZkLTk3MjU1MmViNjUxMyIsInJlc291cmNlR3JvdXBOYW1lIjoiZ2VzdGFsdG1sIiwiYWNjb3VudE5hbWUiOiJmYXN0YWkyIiwid29ya3NwYWNlSWQiOiI4NDY1OWIzNi1hOTllLTRhMTYtYTg3NS03YWU1MmI0Mjg4ZGMifSwibW9kZWxzIjp7fSwibW9kZWxzSW5mbyI6e319 | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in c89ebeb1bfb7
 ---> 69934aebaa9d
Step 4/5 : RUN mv '/var/azureml-app/tmpv1cr_rmp.py' /var/azureml-app/main.py
 ---> Running in 4ba791ea1c0e
 ---> 43e7

WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}

In [ ]:
print('Local service port: {}'.format(local_service.port))

## Check Status and Get Container Logs


In [28]:
print(local_service.get_logs())

2020-09-22T19:19:14,569747500+00:00 - gunicorn/run 
2020-09-22T19:19:14,570339200+00:00 - rsyslog/run 
2020-09-22T19:19:14,570660200+00:00 - iot-server/run 
2020-09-22T19:19:14,577043300+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

## Test Web Service

Call the web service with some input data to get a prediction.

In [29]:
import json

sample_input = 'Testing Data'

print(local_service.run(sample_input))

Checking container health...
Local webservice is running at http://localhost:6789
Test,Positive


## Reload Service

You can update your score.py file and then call `reload()` to quickly restart the service. This will only reload your execution script and dependency files, it will not rebuild the underlying Docker image. As a result, `reload()` is fast, but if you do need to rebuild the image -- to add a new Conda or pip package, for instance -- you will have to call `update()`, instead (see below).

In [ ]:
%%writefile source_directory/x/y/score_template.py
import joblib
import json
import numpy as np
import os

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sklearn_regression_local.pkl')
    # Deserialize the model file back into a sklearn model.
    model = joblib.load(model_path)

    global name, from_location
    # Note here, the entire source directory from inference config gets added into image.
    # Below is an example of how you can use any extra files in image.
    with open('source_directory/extradata.json') as json_file:  
        data = json.load(json_file)
        name = data["people"][0]["name"]
        from_location = data["people"][0]["from"]

input_sample = np.array([[10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0]])
output_sample = np.array([3726.995])

@input_schema('data', NumpyParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        result = model.predict(data)
        # You can return any JSON-serializable object.
        return "Hello " + name + " from " + from_location + " here is your result = " + str(result)
    except Exception as e:
        error = str(e)
        return error

In [31]:
local_service.reload()
print("--------------------------------------------------------------")


Container has been successfully cleaned up.


error: (232, 'WriteFile', 'The pipe is being closed.')

In [27]:
print(local_service.get_logs())

2020-09-22T19:19:14,569747500+00:00 - gunicorn/run 
2020-09-22T19:19:14,570339200+00:00 - rsyslog/run 
2020-09-22T19:19:14,570660200+00:00 - iot-server/run 
2020-09-22T19:19:14,577043300+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

In [ ]:
# After calling reload(), run() will return the updated message.
local_service.run('Testing')


## Update Service

If you want to change your model(s), Conda dependencies, or deployment configuration, call `update()` to rebuild the Docker image.

```python

local_service.update(models=[SomeOtherModelObject],
                     deployment_config=local_config,
                     inference_config=inference_config)
```

In [ ]:
local_service.update(inference_config=inference_config)

## Delete Service

In [ ]:
local_service.delete()